# Homework 3
Due 9/20 @ 11pm:

Create a longitudinal model of a tilt rotor aircraft. You should use the model on pg. 182 
of Stevens and Lewis and add the tilt rotor dynamic by creating a 5th input, to 
represent the tilt of the motors.

Trim the aircraft at 500 ft/s and at 100 ft/s and design a pitch rate controller for both 
trim conditions.

What challenges do you encounter with regard to the aerodynamics at 
the 100 ft/s trim condition?

In [35]:
import casadi as ca
import numpy as np
import control
from casadi.tools.graph import dotgraph
from IPython.display import Image

def draw_graph(expr):
    return Image(dotgraph(expr).create_png())

## Function Definitions

## State Vector
x = true velocity, angle of attack, pitch attitude, pitch rate, altitude, horizontal position

## Input Vector
u = throttle, elevator angle, position of center of gravity (fraction of chord), landing gear extended (true/false)

In [41]:
# Atmosphere function of true velocity and altitude.
# Returns Mach number and dynamic pressure
def ADC(VT, ALT):
    R0 = 2.377E-3                # Sea level density
    TFAC = 1.0 - 0.703E-5*ALT
    T = ca.if_else(ALT > 35000.0, 390.0, 519.0 + TFAC) # Temperature
    RHO = R0 * TFAC**4.14        # Density
    GAM = 1.4
    R = 1716.3
    
    AMACH = VT/ca.sqrt(GAM*R*T)
    QBAR = 0.5*RHO*VT**2
    return(AMACH, QBAR)

def rhs(x, u):

    # Definitions
    S = 2170      # Area of wing
    CBAR = 17.5   # Average chord
    MASS = 5.0E3  # Mass
    IYY = 4.1E6   # Moment of Inertia about y axis
    TSTAT = 6.0E4 # Static thrust
    DTDV = -38.0  # Change in thrust w.r.t. velocity
    ZE = 2.0      # 
    CDCLS = 0.042 # 

    # Per degree
    CLA = 0.085   # Slope of lift curve (1/deg)
    CMA = -0.022  # Slope of moment curve (1/deg)
    CMDE = -0.016 # 

    # Per radian
    CMQ = -16.0   # Coefficient of moment about quarter chord
    CMADOT = -6.0 # Change in coefficient of moment
    CLADOT = 0.0  # Change in coefficient of lift

    RTOD = 57.29578 # Radians to degrees conversion
    GD = 32.17      # 

    # Assigning input vector
    THTL = u[0]
    ELEV = u[1]
    XCG = u[2]
    LAND = u[3]

    # Assigning state vector
    VT = x[0]
    ALPHA = RTOD * x[1] # Angle of attack in degrees
    THETA = x[2]
    Q = x[3]
    H = x[4]

    [MACH, QBAR] = ADC(VT, H)
    QS = QBAR*S
    SALP = ca.sin(x[1])
    CALP = ca.cos(x[1])
    GAM = THETA - x[1]
    SGAM = ca.sin(GAM)
    CGAM = ca.cos(GAM)

    # Correcting for landing
    coeff = ca.if_else(LAND == 0,
      [0.20, 0.016, 0.05, 0.0, 0.0],
      ca.if_else(LAND == 1,
                [1.0, 0.08, -0.20, 0.02, -0.05],
                99))
    if(coeff[0] == 99):
        print('Landing Gear & Flaps?')
    else:
        CLO = coeff[0]
        CDO = coeff[1]
        CMO = coeff[2]
        DCDG = coeff[3]
        DCMG = coeff[4]

    THR = (TSTAT + VT*DTDV) * ca.if_else(THTL < 0, THTL, 0) # Thrust
    CL = CLO + CLA*ALPHA                                    # Lift Coefficient
    CD = DCDG + CDO + CDCLS*CL**2                           # Drag Coefficient

    # Moment Coefficient
    CM = DCMG + CMO + CMA*ALPHA + CMDE + ELEV + CL*(XCG - 0.25)
    
    # Time derivative of state equation
    xd = ca.SX.sym('xd',6)
    xd[0] = (THR*CALP - QS*CD)/MASS - GD*SGAM
    xd[1] = (-THR*SALP - QS*CL + MASS*(VT*Q + GD*CGAM))/(MASS*VT + QS*CLADOT)
    xd[2] = Q
    
    D = 0.5*CBAR*(CMQ*Q + CMADOT*xd[1])/VT
    
    xd[3] = (QS*CBAR*(CM + D) + THR*ZE)/IYY
    xd[4] = VT*SGAM
    xd[5] = VT*CGAM
    return xd

In [42]:
def solve_problem():
    s = ca.SX.sym('s', 3)
    nlp = {'x': s, 'f': objective(s, vt=500, h=0, gamma=0)}
    S = ca.nlpsol('S', 'ipopt', nlp)

    # s = [thtl, elev_deg, alpha]
    s0 = [0.293, 2.46, np.deg2rad(0.58)]
    res = S(x0=s0)
    return res['x']

In [43]:
def constrain(s, vt, h, gamma):
    
    # s is our design vector:
    # s = [thtl, elev_deg, alpha]
    thtl = s[0]
    elev_deg = s[1]
    alpha = s[2]
    
    pos = 0     # we don't care what horiz. position we are at
    q = 0       # we don't want to be rotating, so no pitch-rate
    xcg = 0.25  # we assume xcg at 1/4 chord
    land = 0    # we assume we do not have flaps/gear deployed
    theta = alpha + gamma
    
    # vt, alpha, theta, q, h, pos
    x = ca.vertcat(vt, alpha, theta, q, h, pos)
    
    # thtl, elev_deg, xcg, land
    u = ca.vertcat(thtl, elev_deg, xcg, land)
    return x, u

def trim_cost(x, u):
    x_dot = rhs(x, u)
    return x_dot[0]**2 + 100*x_dot[1]**2 + 10*x_dot[3]**2

def objective(s, vt, h, gamma):
    x, u = constrain(s, vt, h, gamma)
    return trim_cost(x, u)

def trim(vt, h, gamma):
    s = ca.SX.sym('s', 3)
    nlp = {'x': s, 'f': objective(s, vt=vt, h=h, gamma=gamma)}
    S = ca.nlpsol('S', 'ipopt', nlp, {
        'print_time': 0,
        'ipopt': {
            'sb': 'yes',
            'print_level': 0,
            }
        })
    # s = [thtl, elev_deg, alpha]
    s0 = [0.293, -0.0658351, -0.0293941]
    res = S(x0=s0, lbg=0, ubg=0, lbx=[0, -60, -np.deg2rad(5)], ubx=[1, 60, np.deg2rad(18)])
    trim_cost = res['f']
    if trim_cost > 1e-5:
        raise ValueError('Trim failed to converge', trim_cost)
    assert np.abs(float(res['f'])) < 1e-5
    s_opt = res['x']
    x0, u0 = constrain(s_opt, vt, h, gamma)
    return {
        'x0': np.array(x0).reshape(-1),
        'u0': np.array(u0).reshape(-1)
    }

def linearize(trim):
    x0 = trim['x0']
    u0 = trim['u0']
    x = ca.SX.sym('x', 6)
    u = ca.SX.sym('u', 4)
    y = x
    A = ca.jacobian(rhs(x, u), x)
    B = ca.jacobian(rhs(x, u), u)
    C = ca.jacobian(y, x)
    D = ca.jacobian(y, u)
    f_ss = ca.Function('ss', [x, u], [A, B, C, D])
    return control.ss(*f_ss(x0, u0))

## Controller

In [44]:
def pitch_rate_control_design(vt, H, xlim, ylim):
    trim_state = trim(vt=vt, h=0, gamma=0)
    print(trim_state)
    sys = linearize(trim_state)
    G = control.minreal(control.tf(sys[3, 1]), 1e-2)
    control.rlocus(G*H, kvect=np.linspace(0, 1, 1000), xlim=xlim, ylim=ylim);
    Go = G*H
    Gc = control.feedback(Go)
    plt.plot([0, -3], [0, 3*np.arccos(0.707)], '--')
    #plt.axis('equal')
    plt.grid()
    
    plt.figure()
    control.bode(Go, margins=True, dB=True, Hz=True, omega_limits=[1e-2, 1e2], omega_num=1000);
    plt.grid()
    
    plt.figure()
    t, y = control.step_response(Gc, T=np.linspace(0, 3, 1000))
    plt.plot(t, np.real(y))
    plt.grid()
    plt.xlabel('t')
    plt.ylabel('pitch rate, rad/s')


## Analysis

In [45]:
s = control.tf([1, 0], [0, 1])
# this is a PID controller with an extra pole at the origin
pitch_rate_control_design(500, -900*((s + 2 + 1j)*(s + 2 - 1j)/s**2), [-8, 2], [-4, 4])

ValueError: ('Trim failed to converge', DM(4.57627))